라이브러리 임포트

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [ ]:
# 크롤링 정보를 저장할 데이터프레임 선언

data = pd.DataFrame(columns=['스테이션명',	'위치',	'상태정보',	'위도', '경도'])

url_tashu = 'http://www.tashu.or.kr/userStationAction.do?process=list'

# 페이지 이동 시 추가로 적어줘야 되는 url
url_add = '&page='

# 전체 페이지수
total_page = 27

# 27 page에 대한 정보를 추출하는 for문 
for index in range(1, total_page+1):
  if index == 1:
    url = url_tashu # 첫번째 페이지
  else:
    url = url_tashu + url_add + str(index) # 페이지 정보 추가

  # url에서 html 추출하기
  html = urlopen(url)

  # bs4를 이용해 html을 파싱하여 데이터로 변환하기
  bsObject = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')

  # history table 이라고 하는 클래스를 가진 table만 추출하기
  table = bsObject.find_all('table', {'class', 'history_table'})

  # table 내에서 tr 추출
  tr = table[0].find_all('tr')

  # 첫 tr은 contents가 아니라 데이터 정보를 의미하기에 제외시켜줘야 한다.
  tr= tr[1:len(tr)]

  # 한 페이지 씩 정보를 크롤링하기
  for index_tr in range(0, len(tr)):
    td = tr[index_tr].find_all('td')

    # 장소
    station = td[0].text.split('.')[1] # 1. 무역 전시관입구(택시승강장)
    # 위치
    location = td[1].text
    # 상태
    condition = td[2].text

    # 좌표추출
    geo = td[3].a.attrs['href'].split('(')[1] 
    # '36.374580', '127.386131');" 추출했음

    # 위도, 경도 추출을 위한 데이터 클렌징
    geo = re.sub('[;)]', '', geo) 
    # '36.374580', '127.386131');" -> '36.374580', '127.386131'
    detail_location = geo.replace("'", "") # 작은 따옴표 삭제하기

    # 위도 Latitude
    lat = detail_location.split(',')[0]
    # 경도 Longitude
    lon = detail_location.split(',')[1]

    # data에 크롤링 결과를 저장하기
    data = data.append({'스테이션명' : station,
                         '위치' : location,
                         '상태정보': condition,
                         '위도' : lat,
                         '경도' : lon},
                         ignore_index = True)
    print('Complete of' + str(index_tr+1) + 'information with' + str(index) + 'page')

  #크롤링 막히는 것 방지
  time.sleep(1)

print('-' * 77)
print(data)
    

Complete of1information with1page
Complete of2information with1page
Complete of3information with1page
Complete of4information with1page
Complete of5information with1page
Complete of6information with1page
Complete of7information with1page
Complete of8information with1page
Complete of9information with1page
Complete of10information with1page
Complete of1information with2page
Complete of2information with2page
Complete of3information with2page
Complete of4information with2page
Complete of5information with2page
Complete of6information with2page
Complete of7information with2page
Complete of8information with2page
Complete of9information with2page
Complete of10information with2page
Complete of1information with3page
Complete of2information with3page
Complete of3information with3page
Complete of4information with3page
Complete of5information with3page
Complete of6information with3page
Complete of7information with3page
Complete of8information with3page
Complete of9information with3page
Complete of1

크롤링 저장하기

In [ ]:
data.to_csv('TASHU.csv', encoding='utf-8-sig')